# 🎯 Part 7.7: 合规者分析 - CACE 与 LATE 深入解析

## 学习目标

完成本练习后，你将能够：

1. **理解四种合规类型**：Compliers、Always-takers、Never-takers、Defiers
2. **掌握 LATE/CACE 的含义**：局部平均处理效应针对的是谁？
3. **实现工具变量估计**：2SLS 估计及其与合规者的关系
4. **诊断合规者特征**：识别和描述谁是合规者

## 🎬 真实场景：在线教育的课程推送实验

你是某在线教育平台的数据分析师，正在评估"AI 课程推送"对用户学习的影响。

### 实验设计

```
📱 实验方案：
├── 随机分配：50% 用户收到推送通知 (Z=1)
├── 处理定义：是否实际观看了推荐课程 (D)
└── 结果变量：30 天内学习时长 (Y)
```

### 问题：非完美合规

现实中，用户对推送的反应各不相同：

```
👤 小明（Complier 合规者）:
   "收到推送就去看，没收到就不看"
   Z=1 → D=1, Z=0 → D=0 ✅

👤 小红（Always-taker 始终接受者）:
   "我本来就是学霸，不管有没有推送都会主动学"
   Z=1 → D=1, Z=0 → D=1

👤 小刚（Never-taker 始终拒绝者）:
   "推送？关掉！课程？不看！"
   Z=1 → D=0, Z=0 → D=0

👤 小华（Defier 逆反者）:
   "你越推我越不想看，你不推我反而好奇"
   Z=1 → D=0, Z=0 → D=1 😈
```

### 关键问题

1. **ITT vs TOT**：直接比较分组均值（ITT）低估了真实效应吗？
2. **谁的效应？**：我们估计的效应代表哪些用户？
3. **合规者是谁？**：如何识别和描述合规者群体？

## 📐 理论框架：合规类型与 LATE

### 四种合规类型

在工具变量框架下，根据潜在处理状态 $(D_i(0), D_i(1))$ 可以将个体分为四类：

| 类型 | 定义 | $D(0)$ | $D(1)$ | 含义 |
|------|------|--------|--------|------|
| **Compliers** (合规者) | 随工具变化 | 0 | 1 | 推送→看，不推→不看 |
| **Always-takers** | 总是接受 | 1 | 1 | 不管推不推都看 |
| **Never-takers** | 总是拒绝 | 0 | 0 | 不管推不推都不看 |
| **Defiers** (逆反者) | 逆向反应 | 1 | 0 | 推送→不看，不推→看 |

其中 $D(z)$ 表示当工具变量取值为 $z$ 时的潜在处理状态。

### 核心假设：单调性 (Monotonicity)

$$D_i(1) \geq D_i(0) \text{ for all } i$$

**含义**：不存在 Defiers。工具变量只能"鼓励"接受处理，不能"阻止"。

**直觉**：推送只会增加（或不改变）观看概率，不会减少。

### LATE/CACE 定义

**Local Average Treatment Effect (局部平均处理效应)**：

$$\text{LATE} = E[Y_i(1) - Y_i(0) | \text{Complier}] = \frac{E[Y|Z=1] - E[Y|Z=0]}{E[D|Z=1] - E[D|Z=0]}$$

也称为 **CACE (Complier Average Causal Effect)**。

| 符号 | 含义 |
|------|------|
| $Y(d)$ | 处理状态为 $d$ 时的潜在结果 |
| $D(z)$ | 工具变量为 $z$ 时的潜在处理 |
| LATE | 合规者的平均处理效应 |
| ITT | 意向处理效应 = $E[Y|Z=1] - E[Y|Z=0]$ |

### 为什么是合规者的效应？

```
处理组 (Z=1):              对照组 (Z=0):
┌────────────────┐        ┌────────────────┐
│ Compliers: D=1 │        │ Compliers: D=0 │  ← 只有这组变了！
│ Always: D=1    │        │ Always: D=1    │  ← 一样
│ Never: D=0     │        │ Never: D=0     │  ← 一样
└────────────────┘        └────────────────┘

ITT = (Compliers 的效应 × 合规率) + 0 + 0
    = LATE × P(Complier)

所以：LATE = ITT / P(Complier) = ITT / (First Stage)
```

## 🔧 环境准备

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# 颜色方案
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'neutral': '#828282',
    'purple': '#9B51E0',
    'complier': '#27AE60',
    'always_taker': '#2D9CDB',
    'never_taker': '#EB5757'
}

print("✅ 环境准备完成！")

## 📊 数据生成：在线教育实验

In [ ]:
def generate_compliance_data(
    n_samples: int = 5000,
    complier_rate: float = 0.4,      # 合规者比例
    always_taker_rate: float = 0.2,  # 始终接受者比例
    never_taker_rate: float = 0.4,   # 始终拒绝者比例
    true_late: float = 15,           # 合规者的真实效应（学习时长增加）
    seed: int = 42
) -> pd.DataFrame:
    """
    生成带有合规类型的实验数据
    
    场景：在线教育课程推送实验
    - Z: 是否收到推送（随机分配）
    - D: 是否观看推荐课程（可能不合规）
    - Y: 30 天学习时长（小时）
    
    参数:
        n_samples: 样本量
        complier_rate: 合规者比例
        always_taker_rate: 始终接受者比例
        never_taker_rate: 始终拒绝者比例
        true_late: 合规者的真实处理效应
    
    返回:
        DataFrame
    """
    np.random.seed(seed)
    
    # 用户特征
    age = np.random.normal(28, 8, n_samples).clip(18, 60)
    education = np.random.choice([1, 2, 3, 4], n_samples, p=[0.2, 0.3, 0.35, 0.15])  # 学历
    prior_engagement = np.random.exponential(5, n_samples)  # 历史参与度
    
    # 合规类型（与特征相关）
    # 年轻、高学历、高参与度的人更可能是合规者或始终接受者
    compliance_score = (
        -0.02 * (age - 28) +           # 年轻人更可能合规
        0.2 * (education - 2) +         # 高学历更可能响应
        0.05 * prior_engagement         # 高参与度更可能响应
    )
    compliance_prob = 1 / (1 + np.exp(-compliance_score))
    
    # 分配合规类型
    compliance_type = np.empty(n_samples, dtype='<U15')
    
    for i in range(n_samples):
        # 根据特征调整各类型概率
        p_complier = complier_rate * (0.5 + 0.5 * compliance_prob[i])
        p_always = always_taker_rate * (0.3 + 0.7 * compliance_prob[i])
        p_never = 1 - p_complier - p_always
        p_never = max(0.1, p_never)  # 确保非负
        
        # 归一化
        total = p_complier + p_always + p_never
        probs = [p_complier/total, p_always/total, p_never/total]
        
        compliance_type[i] = np.random.choice(
            ['complier', 'always_taker', 'never_taker'],
            p=probs
        )
    
    # 随机分配工具变量（推送）
    Z = np.random.binomial(1, 0.5, n_samples)
    
    # 实际处理状态（根据合规类型和 Z 决定）
    D = np.zeros(n_samples, dtype=int)
    D[compliance_type == 'always_taker'] = 1
    D[(compliance_type == 'complier') & (Z == 1)] = 1
    # never_taker 和 complier with Z=0 保持 D=0
    
    # 潜在结果
    # Y(0): 基础学习时长（与特征相关）
    Y0 = (
        10 +                           # 基础
        0.1 * (age - 28) +             # 年龄效应
        2 * (education - 2) +          # 学历效应
        0.5 * prior_engagement +       # 参与度效应
        np.random.normal(0, 5, n_samples)
    ).clip(0, 100)
    
    # Y(1): 异质性处理效应
    # 合规者有正效应，其他人没有（因为行为本来就不变）
    treatment_effect = np.zeros(n_samples)
    
    # 合规者的异质效应
    complier_mask = compliance_type == 'complier'
    treatment_effect[complier_mask] = (
        true_late +
        0.5 * (30 - age[complier_mask]) / 10 +  # 年轻人效应更大
        np.random.normal(0, 3, complier_mask.sum())
    )
    
    Y1 = Y0 + treatment_effect
    
    # 观察结果
    Y = np.where(D == 1, Y1, Y0)
    
    # 构建 DataFrame
    df = pd.DataFrame({
        'user_id': range(n_samples),
        'age': age,
        'education': education,
        'prior_engagement': prior_engagement,
        'compliance_type': compliance_type,
        'Z': Z,                    # 工具变量（推送分配）
        'D': D,                    # 处理状态（实际观看）
        'Y': Y,                    # 观察结果
        'Y0': Y0,                  # 潜在结果（未处理）
        'Y1': Y1,                  # 潜在结果（处理）
        'true_effect': treatment_effect
    })
    
    return df, {
        'true_late': true_late,
        'complier_rate': (compliance_type == 'complier').mean(),
        'always_taker_rate': (compliance_type == 'always_taker').mean(),
        'never_taker_rate': (compliance_type == 'never_taker').mean()
    }

# 生成数据
df, true_params = generate_compliance_data(n_samples=5000)

print(f"📊 数据生成完成！")
print(f"\n合规类型分布：")
print(df['compliance_type'].value_counts(normalize=True).to_string())
print(f"\n真实 LATE: {true_params['true_late']:.1f} 小时")

## 📈 可视化：理解合规类型

In [ ]:
# 合规类型分布
type_counts = df['compliance_type'].value_counts()

fig = go.Figure(data=[
    go.Pie(
        labels=['合规者 (Compliers)', '始终接受者 (Always-takers)', '始终拒绝者 (Never-takers)'],
        values=[type_counts['complier'], type_counts['always_taker'], type_counts['never_taker']],
        marker_colors=[COLORS['complier'], COLORS['always_taker'], COLORS['never_taker']],
        hole=0.4,
        textinfo='label+percent'
    )
])

fig.update_layout(
    title='👥 用户合规类型分布',
    template='plotly_white',
    height=400
)

fig.show()

In [ ]:
# 按 Z 和合规类型展示 D 的分布
cross_tab = pd.crosstab(
    [df['compliance_type'], df['Z']],
    df['D'],
    normalize='index'
) * 100

print("📊 按合规类型和推送状态的观看率：")
print(cross_tab.round(1))

In [ ]:
# 可视化：处理状态与工具变量的关系（按合规类型）
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('合规者', '始终接受者', '始终拒绝者')
)

for i, ctype in enumerate(['complier', 'always_taker', 'never_taker'], 1):
    sub_df = df[df['compliance_type'] == ctype]
    
    # 按 Z 分组计算 D 的均值
    d_by_z = sub_df.groupby('Z')['D'].mean()
    
    fig.add_trace(
        go.Bar(
            x=['未推送 (Z=0)', '推送 (Z=1)'],
            y=[d_by_z.get(0, 0), d_by_z.get(1, 0)],
            marker_color=[COLORS['neutral'], COLORS['primary']],
            text=[f'{d_by_z.get(0, 0):.1%}', f'{d_by_z.get(1, 0):.1%}'],
            textposition='outside'
        ),
        row=1, col=i
    )

fig.update_layout(
    title='📊 不同合规类型对推送的反应',
    height=350,
    template='plotly_white',
    showlegend=False
)

fig.update_yaxes(title_text='观看率', range=[0, 1.1])

fig.show()

print("\n💡 观察：")
print("   • 合规者：推送大幅提高观看率（从 0% 到 100%）")
print("   • 始终接受者：不管推不推都看（100%）")
print("   • 始终拒绝者：不管推不推都不看（0%）")

## 🛠️ 估计方法比较

### 方法 1: 朴素 OLS（有偏）

In [ ]:
# 朴素 OLS：直接回归 Y ~ D
X_naive = sm.add_constant(df['D'])
model_naive = sm.OLS(df['Y'], X_naive).fit()

print("📊 方法 1: 朴素 OLS (Y ~ D)")
print(f"   估计效应: {model_naive.params['D']:.2f} 小时")
print(f"   真实 LATE: {true_params['true_late']:.2f} 小时")
print(f"   偏差: {model_naive.params['D'] - true_params['true_late']:.2f}")
print("\n   ⚠️ 朴素 OLS 有偏！因为 D 与未观测因素相关（始终接受者本身学习意愿就高）")

### 方法 2: ITT（意向处理效应）

In [ ]:
# ITT: 比较 Z=1 和 Z=0 组的结果均值
itt = df.groupby('Z')['Y'].mean()
itt_effect = itt[1] - itt[0]

print("📊 方法 2: ITT (意向处理效应)")
print(f"   推送组均值 (Z=1): {itt[1]:.2f} 小时")
print(f"   对照组均值 (Z=0): {itt[0]:.2f} 小时")
print(f"   ITT: {itt_effect:.2f} 小时")
print(f"\n   💡 ITT 是无偏的，但估计的是'被分配到推送'的效应，不是'实际观看'的效应")
print(f"   由于不完美合规，ITT < LATE")

### 方法 3: 2SLS / Wald 估计量（LATE）

In [ ]:
# ============================================================
# 🎯 练习 1：实现 Wald 估计量
# ============================================================
# 目标：计算 LATE (局部平均处理效应)
# 提示：
#   1. First Stage: E[D|Z=1] - E[D|Z=0] = 合规率
#   2. Reduced Form: E[Y|Z=1] - E[Y|Z=0] = ITT
#   3. LATE = ITT / First Stage
# ============================================================

def wald_estimator(df: pd.DataFrame) -> dict:
    """
    计算 Wald 估计量（LATE）
    
    参数:
        df: 包含 Z, D, Y 的 DataFrame
    
    返回:
        包含估计结果的字典
    """
    # === 你的代码开始 ===
    
    # 1. 按 Z 分组计算 D 和 Y 的均值
    
    # 2. 计算 First Stage: P(D=1|Z=1) - P(D=1|Z=0)
    
    # 3. 计算 Reduced Form (ITT): E[Y|Z=1] - E[Y|Z=0]
    
    # 4. 计算 LATE = ITT / First Stage
    
    pass  # 删除此行，填入你的代码
    
    # === 你的代码结束 ===
    
    return {
        'first_stage': None,
        'itt': None,
        'late': None
    }

<details>
<summary>💡 点击查看参考答案</summary>

```python
def wald_estimator(df: pd.DataFrame) -> dict:
    """
    计算 Wald 估计量（LATE）
    """
    # 按 Z 分组计算均值
    by_z = df.groupby('Z').agg({'D': 'mean', 'Y': 'mean'})
    
    # First Stage: 推送对观看的因果效应
    first_stage = by_z.loc[1, 'D'] - by_z.loc[0, 'D']
    
    # Reduced Form (ITT): 推送对学习时长的因果效应
    itt = by_z.loc[1, 'Y'] - by_z.loc[0, 'Y']
    
    # LATE = ITT / First Stage
    late = itt / first_stage if first_stage != 0 else np.nan
    
    return {
        'first_stage': first_stage,
        'itt': itt,
        'late': late,
        'd_z1': by_z.loc[1, 'D'],
        'd_z0': by_z.loc[0, 'D'],
        'y_z1': by_z.loc[1, 'Y'],
        'y_z0': by_z.loc[0, 'Y']
    }
```

**解析**：
- **First Stage**: 工具变量 Z 对处理变量 D 的因果效应，等于合规率
- **Reduced Form (ITT)**: 工具变量 Z 对结果变量 Y 的总效应
- **LATE**: 两者的比值，即合规者的平均处理效应

</details>

In [ ]:
# 执行估计
wald_result = wald_estimator(df)

print("📊 方法 3: Wald 估计量 (LATE)")
print(f"\n   First Stage (合规率):")
print(f"      P(D=1|Z=1) = {wald_result['d_z1']:.3f}")
print(f"      P(D=1|Z=0) = {wald_result['d_z0']:.3f}")
print(f"      差值 = {wald_result['first_stage']:.3f}")
print(f"\n   Reduced Form (ITT):")
print(f"      E[Y|Z=1] = {wald_result['y_z1']:.2f}")
print(f"      E[Y|Z=0] = {wald_result['y_z0']:.2f}")
print(f"      差值 = {wald_result['itt']:.2f}")
print(f"\n   ⭐ LATE = ITT / First Stage = {wald_result['late']:.2f} 小时")
print(f"   ⭐ 真实 LATE: {true_params['true_late']:.2f} 小时")

### 方法 4: 2SLS 回归

In [ ]:
# ============================================================
# 🎯 练习 2：实现两阶段最小二乘法 (2SLS)
# ============================================================
# 目标：使用 2SLS 方法估计 LATE
# 提示：
#   1. 第一阶段：D ~ Z (+ 控制变量)
#   2. 获取预测值 D_hat
#   3. 第二阶段：Y ~ D_hat (+ 控制变量)
#   4. 使用 statsmodels 的 IV2SLS 获得正确的标准误
# ============================================================

def two_stage_least_squares(
    df: pd.DataFrame,
    controls: list = None
) -> dict:
    """
    两阶段最小二乘法估计
    
    参数:
        df: 包含 Z, D, Y 和控制变量的 DataFrame
        controls: 控制变量列表
    
    返回:
        估计结果字典
    """
    # === 你的代码开始 ===
    
    # 1. 准备数据（工具变量和控制变量）
    
    # 2. 第一阶段：用 Z（和控制变量）预测 D
    
    # 3. 使用 statsmodels 的 IV2SLS 获得正确的标准误
    
    pass  # 删除此行，填入你的代码
    
    # === 你的代码结束 ===
    
    return {
        'late': None,
        'se': None,
        'first_stage_f': None
    }

<details>
<summary>💡 点击查看参考答案</summary>

```python
def two_stage_least_squares(
    df: pd.DataFrame,
    controls: list = None
) -> dict:
    """
    两阶段最小二乘法估计
    """
    if controls is None:
        controls = []
    
    # 准备数据
    if controls:
        X_first = sm.add_constant(df[['Z'] + controls])
        X_second_base = df[controls].copy()
    else:
        X_first = sm.add_constant(df[['Z']])
        X_second_base = pd.DataFrame()
    
    # 第一阶段：D ~ Z + controls
    first_stage = sm.OLS(df['D'], X_first).fit()
    D_hat = first_stage.predict(X_first)
    
    # 第一阶段 F 统计量
    first_stage_f = first_stage.fvalue
    
    # 使用 statsmodels 的 IV2SLS（标准误更准确）
    if controls:
        endog = df['Y']
        exog = sm.add_constant(df[['D'] + controls])
        instrument = sm.add_constant(df[['Z'] + controls])
    else:
        endog = df['Y']
        exog = sm.add_constant(df['D'])
        instrument = sm.add_constant(df['Z'])
    
    iv_model = IV2SLS(endog, exog, instrument).fit()
    
    return {
        'late': iv_model.params['D'],
        'se': iv_model.bse['D'],
        'ci_lower': iv_model.params['D'] - 1.96 * iv_model.bse['D'],
        'ci_upper': iv_model.params['D'] + 1.96 * iv_model.bse['D'],
        'first_stage_f': first_stage_f,
        'first_stage_coef': first_stage.params['Z'],
        'n_obs': len(df)
    }
```

**解析**：
- **第一阶段**: 用工具变量 Z 预测处理变量 D，检查工具变量的强度（F-stat）
- **IV2SLS**: 使用 statsmodels 的专用函数得到正确调整的标准误
- **First Stage F-stat**: F > 10 表示工具变量足够强，避免弱工具变量问题

</details>

In [ ]:
# 不含控制变量
iv_result = two_stage_least_squares(df)

print("📊 方法 4: 2SLS 估计")
print(f"\n   ⭐ LATE: {iv_result['late']:.2f} ± {1.96 * iv_result['se']:.2f}")
print(f"   95% CI: [{iv_result['ci_lower']:.2f}, {iv_result['ci_upper']:.2f}]")
print(f"   真实 LATE: {true_params['true_late']:.2f}")
print(f"\n   First Stage F-stat: {iv_result['first_stage_f']:.1f}")
print(f"   (F > 10 表示工具变量足够强)")

In [ ]:
# 📝 参考答案
# ================================

def two_stage_least_squares(
    df: pd.DataFrame,
    controls: list = None
) -> dict:
    """
    两阶段最小二乘法估计
    """
    if controls is None:
        controls = []
    
    # 准备数据
    if controls:
        X_first = sm.add_constant(df[['Z'] + controls])
        X_second_base = df[controls].copy()
    else:
        X_first = sm.add_constant(df[['Z']])
        X_second_base = pd.DataFrame()
    
    # 第一阶段：D ~ Z + controls
    first_stage = sm.OLS(df['D'], X_first).fit()
    D_hat = first_stage.predict(X_first)
    
    # 第一阶段 F 统计量
    first_stage_f = first_stage.fvalue
    
    # 第二阶段：Y ~ D_hat + controls
    if controls:
        X_second = sm.add_constant(pd.concat([pd.Series(D_hat, name='D_hat'), X_second_base], axis=1))
    else:
        X_second = sm.add_constant(pd.DataFrame({'D_hat': D_hat}))
    
    second_stage = sm.OLS(df['Y'], X_second).fit()
    
    # 注意：手动 2SLS 的标准误需要调整，这里使用 statsmodels 的 IV2SLS
    # 使用 statsmodels 的 IV 估计（更准确的标准误）
    if controls:
        endog = df['Y']
        exog = sm.add_constant(df[['D'] + controls])
        instrument = sm.add_constant(df[['Z'] + controls])
    else:
        endog = df['Y']
        exog = sm.add_constant(df['D'])
        instrument = sm.add_constant(df['Z'])
    
    iv_model = IV2SLS(endog, exog, instrument).fit()
    
    return {
        'late': iv_model.params['D'],
        'se': iv_model.bse['D'],
        'ci_lower': iv_model.params['D'] - 1.96 * iv_model.bse['D'],
        'ci_upper': iv_model.params['D'] + 1.96 * iv_model.bse['D'],
        'first_stage_f': first_stage_f,
        'first_stage_coef': first_stage.params['Z'],
        'n_obs': len(df)
    }

# 不含控制变量
iv_result = two_stage_least_squares(df)

print("📊 方法 4: 2SLS 估计")
print(f"\n   ⭐ LATE: {iv_result['late']:.2f} ± {1.96 * iv_result['se']:.2f}")
print(f"   95% CI: [{iv_result['ci_lower']:.2f}, {iv_result['ci_upper']:.2f}]")
print(f"   真实 LATE: {true_params['true_late']:.2f}")
print(f"\n   First Stage F-stat: {iv_result['first_stage_f']:.1f}")
print(f"   (F > 10 表示工具变量足够强)")

In [ ]:
# 带控制变量的 2SLS
iv_result_controls = two_stage_least_squares(
    df, 
    controls=['age', 'education', 'prior_engagement']
)

print("📊 带控制变量的 2SLS：")
print(f"   LATE: {iv_result_controls['late']:.2f} ± {1.96 * iv_result_controls['se']:.2f}")
print(f"   (加入控制变量可以提高精度）")

## 📊 估计方法对比

In [ ]:
# ------------------------------------------------------------
# 📖 示范：分析合规者特征
# ------------------------------------------------------------
# 说明：由于我们有真实的合规类型标签，可以直接比较各类型的特征分布

def analyze_complier_characteristics(df: pd.DataFrame) -> pd.DataFrame:
    """
    分析合规者特征
    
    方法：
    - 方法 1：直接比较已知合规类型的特征（如果有标签）
    - 方法 2：使用 Abadie (2003) 方法估计合规者特征（下面展示）
    """
    feature_cols = ['age', 'education', 'prior_engagement']
    
    stats = []
    for ctype in ['complier', 'always_taker', 'never_taker']:
        sub_df = df[df['compliance_type'] == ctype]
        row = {'type': ctype, 'n': len(sub_df)}
        for col in feature_cols:
            row[f'{col}_mean'] = sub_df[col].mean()
        stats.append(row)
    
    return pd.DataFrame(stats)

complier_stats = analyze_complier_characteristics(df)
print("📊 各合规类型的特征对比：")
print(complier_stats.to_string(index=False))

## 🔬 合规者特征分析

LATE 是合规者的平均效应。了解合规者是谁，有助于理解效应的外部有效性。

In [ ]:
# TODO 3: 分析合规者特征
# 提示：
# - 方法 1：直接比较已知合规类型的特征（如果有标签）
# - 方法 2：比较 D=1|Z=1 和 D=1|Z=0 人群的特征（Always-takers vs Compliers + Always-takers）

def analyze_complier_characteristics(df: pd.DataFrame) -> pd.DataFrame:
    """
    分析合规者特征
    """
    # 由于我们有真实的合规类型标签，可以直接比较
    feature_cols = ['age', 'education', 'prior_engagement']
    
    stats = []
    for ctype in ['complier', 'always_taker', 'never_taker']:
        sub_df = df[df['compliance_type'] == ctype]
        row = {'type': ctype, 'n': len(sub_df)}
        for col in feature_cols:
            row[f'{col}_mean'] = sub_df[col].mean()
        stats.append(row)
    
    return pd.DataFrame(stats)

complier_stats = analyze_complier_characteristics(df)
print("📊 各合规类型的特征对比：")
print(complier_stats.to_string(index=False))

In [ ]:
# 可视化合规者特征
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('年龄分布', '学历分布', '历史参与度分布')
)

for i, ctype in enumerate(['complier', 'always_taker', 'never_taker']):
    sub_df = df[df['compliance_type'] == ctype]
    color = COLORS[ctype] if ctype in COLORS else COLORS['neutral']
    name = {'complier': '合规者', 'always_taker': '始终接受', 'never_taker': '始终拒绝'}[ctype]
    
    # 年龄
    fig.add_trace(
        go.Histogram(
            x=sub_df['age'],
            name=name,
            marker_color=color,
            opacity=0.6,
            legendgroup=ctype,
            showlegend=(i == 0)
        ),
        row=1, col=1
    )
    
    # 学历
    fig.add_trace(
        go.Histogram(
            x=sub_df['education'],
            name=name,
            marker_color=color,
            opacity=0.6,
            legendgroup=ctype,
            showlegend=False
        ),
        row=1, col=2
    )
    
    # 参与度
    fig.add_trace(
        go.Histogram(
            x=sub_df['prior_engagement'],
            name=name,
            marker_color=color,
            opacity=0.6,
            legendgroup=ctype,
            showlegend=False
        ),
        row=1, col=3
    )

fig.update_layout(
    title='👥 不同合规类型的特征分布',
    barmode='overlay',
    template='plotly_white',
    height=350
)

fig.show()

In [ ]:
# 使用 Abadie (2003) 的方法估计合规者特征（不需要真实标签）
def estimate_complier_mean(df: pd.DataFrame, feature: str) -> float:
    """
    使用 Abadie 方法估计合规者的特征均值
    
    E[X|Complier] = (E[X*D|Z=1] - E[X*D|Z=0]) / (E[D|Z=1] - E[D|Z=0])
    """
    # 分组计算
    z1 = df[df['Z'] == 1]
    z0 = df[df['Z'] == 0]
    
    # E[X*D|Z]
    xd_z1 = (z1[feature] * z1['D']).mean()
    xd_z0 = (z0[feature] * z0['D']).mean()
    
    # E[D|Z]
    d_z1 = z1['D'].mean()
    d_z0 = z0['D'].mean()
    
    # 合规者特征均值
    complier_mean = (xd_z1 - xd_z0) / (d_z1 - d_z0)
    
    return complier_mean

print("📊 Abadie 方法估计的合规者特征：")
for feature in ['age', 'education', 'prior_engagement']:
    est_mean = estimate_complier_mean(df, feature)
    true_mean = df[df['compliance_type'] == 'complier'][feature].mean()
    print(f"   {feature}: 估计 = {est_mean:.2f}, 真实 = {true_mean:.2f}")

## 📦 封装：完整的 LATE 估计器

In [ ]:
class LATEEstimator:
    """
    局部平均处理效应 (LATE/CACE) 估计器
    
    使用工具变量方法估计合规者的平均处理效应
    """
    
    def __init__(self, controls: list = None):
        """
        参数:
            controls: 控制变量列名列表
        """
        self.controls = controls or []
        self.results_ = None
    
    def fit(self, Z: np.ndarray, D: np.ndarray, Y: np.ndarray, 
            X: pd.DataFrame = None) -> 'LATEEstimator':
        """
        拟合 LATE 估计器
        
        参数:
            Z: 工具变量
            D: 处理变量
            Y: 结果变量
            X: 控制变量 DataFrame
        """
        # Wald 估计
        by_z = pd.DataFrame({'Z': Z, 'D': D, 'Y': Y}).groupby('Z').mean()
        
        first_stage = by_z.loc[1, 'D'] - by_z.loc[0, 'D']
        itt = by_z.loc[1, 'Y'] - by_z.loc[0, 'Y']
        wald_late = itt / first_stage if first_stage != 0 else np.nan
        
        # 2SLS 估计
        df_temp = pd.DataFrame({'Z': Z, 'D': D, 'Y': Y})
        if X is not None and self.controls:
            for col in self.controls:
                if col in X.columns:
                    df_temp[col] = X[col].values
        
        # 使用 statsmodels IV2SLS
        if self.controls and X is not None:
            valid_controls = [c for c in self.controls if c in df_temp.columns]
            endog = df_temp['Y']
            exog = sm.add_constant(df_temp[['D'] + valid_controls])
            instrument = sm.add_constant(df_temp[['Z'] + valid_controls])
        else:
            endog = df_temp['Y']
            exog = sm.add_constant(df_temp['D'])
            instrument = sm.add_constant(df_temp['Z'])
        
        iv_model = IV2SLS(endog, exog, instrument).fit()
        
        # First stage F-stat
        if self.controls and X is not None:
            X_first = sm.add_constant(df_temp[['Z'] + valid_controls])
        else:
            X_first = sm.add_constant(df_temp['Z'])
        first_stage_model = sm.OLS(D, X_first).fit()
        
        self.results_ = {
            'wald_late': wald_late,
            'late': iv_model.params['D'],
            'se': iv_model.bse['D'],
            'ci_lower': iv_model.params['D'] - 1.96 * iv_model.bse['D'],
            'ci_upper': iv_model.params['D'] + 1.96 * iv_model.bse['D'],
            'first_stage': first_stage,
            'itt': itt,
            'first_stage_f': first_stage_model.fvalue,
            'n_obs': len(Y),
            'complier_share': first_stage
        }
        
        return self
    
    def get_late(self) -> float:
        """获取 LATE 估计值"""
        return self.results_['late']
    
    def summary(self) -> pd.DataFrame:
        """返回结果摘要"""
        return pd.DataFrame({
            '指标': [
                'LATE (2SLS)',
                '标准误',
                '95% CI 下界',
                '95% CI 上界',
                'ITT',
                'First Stage (合规率)',
                'First Stage F-stat',
                '样本量'
            ],
            '值': [
                f"{self.results_['late']:.3f}",
                f"{self.results_['se']:.3f}",
                f"{self.results_['ci_lower']:.3f}",
                f"{self.results_['ci_upper']:.3f}",
                f"{self.results_['itt']:.3f}",
                f"{self.results_['first_stage']:.3f}",
                f"{self.results_['first_stage_f']:.1f}",
                f"{self.results_['n_obs']}"
            ]
        })
    
    def diagnose_iv(self) -> None:
        """诊断工具变量有效性"""
        print("🔬 工具变量诊断：")
        print(f"\n   1. 相关性 (Relevance):")
        print(f"      First Stage = {self.results_['first_stage']:.3f}")
        print(f"      F-stat = {self.results_['first_stage_f']:.1f}")
        if self.results_['first_stage_f'] > 10:
            print(f"      ✅ F > 10，工具变量足够强")
        else:
            print(f"      ⚠️ F < 10，可能存在弱工具变量问题")
        
        print(f"\n   2. 排他性 (Exclusion):")
        print(f"      需要假设 Z 只通过 D 影响 Y")
        print(f"      （无法从数据中验证，需要领域知识）")
        
        print(f"\n   3. 单调性 (Monotonicity):")
        print(f"      需要假设不存在 Defiers")
        print(f"      （在本场景中：推送不会减少观看意愿）")

# 使用估计器
estimator = LATEEstimator(controls=['age', 'education', 'prior_engagement'])
estimator.fit(
    Z=df['Z'].values,
    D=df['D'].values,
    Y=df['Y'].values,
    X=df[['age', 'education', 'prior_engagement']]
)

print("\n📊 LATE 估计结果：")
print(estimator.summary().to_string(index=False))

print("\n")
estimator.diagnose_iv()

## 💼 业务解读与决策

### 理解 LATE 的含义

In [ ]:
print("📊 结果解读：")
print("="*50)
print(f"\n✅ LATE = {estimator.get_late():.1f} 小时")
print(f"\n这意味着：")
print(f"   对于那些'会被推送影响'的用户（合规者），")
print(f"   观看推荐课程平均增加 {estimator.get_late():.1f} 小时的学习时长")
print(f"\n⚠️ 注意：这个效应只适用于合规者！")
print(f"   合规者占比约 {estimator.results_['complier_share']:.1%}")
print(f"\n对其他人群：")
print(f"   • 始终接受者：不管推不推都会看，推送对他们没有额外价值")
print(f"   • 始终拒绝者：不管推不推都不看，推送对他们无效")

In [ ]:
# 业务决策分析
print("📊 业务决策分析：")
print("="*50)

# 假设的成本收益参数
push_cost = 0.1  # 每次推送成本（元）
hour_value = 5   # 每小时学习时长的价值（元）

n_total = 10000  # 总用户数
complier_share = estimator.results_['complier_share']
late = estimator.get_late()

# 计算 ROI
total_cost = n_total * push_cost
total_benefit = n_total * complier_share * late * hour_value
roi = (total_benefit - total_cost) / total_cost

print(f"\n假设参数：")
print(f"   推送成本: {push_cost} 元/次")
print(f"   学习时长价值: {hour_value} 元/小时")
print(f"   用户数: {n_total:,}")

print(f"\n成本收益分析：")
print(f"   总推送成本: {total_cost:,.0f} 元")
print(f"   预期收益: {total_benefit:,.0f} 元")
print(f"      = {n_total} × {complier_share:.1%} × {late:.1f} × {hour_value}")
print(f"   ROI: {roi:.0%}")

if roi > 0:
    print(f"\n✅ 推送策略是有利可图的！")
else:
    print(f"\n❌ 推送策略可能不划算")

## 💡 思考题

### 基础理解

1. **为什么 LATE 估计的是合规者的效应？** 从数学上解释 Wald 估计量的含义。

2. **单调性假设什么时候可能被违反？** 举一个存在 Defiers 的例子。

### 深入分析

3. **LATE 与 ATE 的关系是什么？** 在什么条件下 LATE = ATE？

4. **如果 First Stage F-stat 很小（如 F < 5），会有什么问题？** 如何解决？

### 实战应用

5. **假设你发现合规者主要是年轻用户，这对产品策略有什么启示？**

6. **在什么业务场景下，你更关心 ITT 而不是 LATE？** 请举例说明。

## 📋 本章小结

| 概念 | 定义 | 关键点 |
|------|------|--------|
| **Compliers** | 响应工具变量的个体 | D(0)=0, D(1)=1 |
| **Always-takers** | 始终接受处理 | D(0)=1, D(1)=1 |
| **Never-takers** | 始终拒绝处理 | D(0)=0, D(1)=0 |
| **Defiers** | 逆向响应 | D(0)=1, D(1)=0（通常假设不存在）|
| **LATE/CACE** | 合规者的平均处理效应 | = ITT / First Stage |
| **ITT** | 意向处理效应 | E[Y|Z=1] - E[Y|Z=0] |
| **First Stage** | 合规率 | E[D|Z=1] - E[D|Z=0] |

### IV 估计的核心假设

| 假设 | 含义 | 验证方法 |
|------|------|----------|
| **相关性** | Z 影响 D | First Stage F-stat > 10 |
| **排他性** | Z 只通过 D 影响 Y | 无法检验，依赖领域知识 |
| **单调性** | 无 Defiers | 通常基于合理性论证 |
| **SUTVA** | 无干扰 | 实验设计保证 |

### 方法对比

| 方法 | 估计的是什么 | 优点 | 缺点 |
|------|-------------|------|------|
| 朴素 OLS | 有偏估计 | 简单 | 存在选择偏差 |
| ITT | 分配效应 | 无偏 | 稀释了真实效应 |
| LATE/2SLS | 合规者效应 | 因果解释 | 只适用于合规者 |

## 📚 延伸阅读

1. **Angrist, J., Imbens, G., & Rubin, D. (1996)** - "Identification of Causal Effects Using Instrumental Variables" - LATE 的经典论文

2. **Imbens, G. & Angrist, J. (1994)** - "Identification and Estimation of Local Average Treatment Effects"

3. **Abadie, A. (2003)** - "Semiparametric Instrumental Variable Estimation of Treatment Response Models" - 合规者特征估计

4. **Angrist, J. & Pischke, J. (2009)** - "Mostly Harmless Econometrics" - 第 4 章关于 IV 的详细讨论